In [1]:
import pandas as pd
import numpy as np
from fancyimpute import KNN

In [2]:
test = pd.read_csv('cs-test.csv')
train = pd.read_csv('cs-training.csv')
print test.shape
print train.shape

(101503, 12)
(150000, 12)


In [3]:
print train.columns

Index([u'Unnamed: 0', u'SeriousDlqin2yrs',
       u'RevolvingUtilizationOfUnsecuredLines', u'age',
       u'NumberOfTime30-59DaysPastDueNotWorse', u'DebtRatio', u'MonthlyIncome',
       u'NumberOfOpenCreditLinesAndLoans', u'NumberOfTimes90DaysLate',
       u'NumberRealEstateLoansOrLines',
       u'NumberOfTime60-89DaysPastDueNotWorse', u'NumberOfDependents'],
      dtype='object')


In [4]:
train = train.drop('Unnamed: 0', axis=1)

In [7]:
# KNN imputation
tmp = train.select_dtypes(include=[np.float]).head(n=100).as_matrix()
tmpFilled = pd.DataFrame(KNN(k=3).complete(tmp))

Imputing row 1/100 with 0 missing, elapsed time: 0.008


In [16]:
print tmpFilled.shape
print tmp.shape
print train.select_dtypes(include=[np.float]).columns

(100, 4)
(100, 4)
Index([u'RevolvingUtilizationOfUnsecuredLines', u'DebtRatio', u'MonthlyIncome',
       u'NumberOfDependents'],
      dtype='object')


In [13]:
print tmp

[[  7.66126609e-01   8.02982129e-01   9.12000000e+03   2.00000000e+00]
 [  9.57151019e-01   1.21876201e-01   2.60000000e+03   1.00000000e+00]
 [  6.58180140e-01   8.51133750e-02   3.04200000e+03   0.00000000e+00]
 [  2.33809776e-01   3.60496820e-02   3.30000000e+03   0.00000000e+00]
 [  9.07239400e-01   2.49256950e-02   6.35880000e+04   0.00000000e+00]
 [  2.13178682e-01   3.75606969e-01   3.50000000e+03   1.00000000e+00]
 [  3.05682465e-01   5.71000000e+03              nan   0.00000000e+00]
 [  7.54463648e-01   2.09940017e-01   3.50000000e+03   0.00000000e+00]
 [  1.16950644e-01   4.60000000e+01              nan              nan]
 [  1.89169052e-01   6.06290901e-01   2.36840000e+04   2.00000000e+00]
 [  6.44225962e-01   3.09476210e-01   2.50000000e+03   0.00000000e+00]
 [  1.87981200e-02   5.31528760e-01   6.50100000e+03   2.00000000e+00]
 [  1.03518570e-02   2.98354075e-01   1.24540000e+04   2.00000000e+00]
 [  9.64672555e-01   3.82964747e-01   1.37000000e+04   2.00000000e+00]
 [  1.

In [18]:
from IPython.display import display
display(tmpFilled)

,0,1,2,3
0,0.766127,0.802982,9120.000000,2.00000
1,0.957151,0.121876,2600.000000,1.00000
2,0.658180,0.085113,3042.000000,0.00000
3,0.233810,0.036050,3300.000000,0.00000
4,0.907239,0.024926,63588.000000,0.00000
5,0.213179,0.375607,3500.000000,1.00000
6,0.305682,5710.000000,0.170694,0.00000
7,0.754464,0.209940,3500.000000,0.00000
8,0.116951,46.000000,6049.642416,0.94580
9,0.189169,0.606291,23684.000000,2.00000
